In [1]:
import math
import random
import matplotlib.pyplot as plt

 Loading of the data set
----------------------------
- The data containing the city and its co-ordinates is stored in a text file named(city_data)
- The first column contains the city names and second and third colum contains the x and y co-ordinates
- load_city function is used to get the data from the text file and store in the list for computaion of genetic algorithm

In [2]:
def load_city():
    city_list=[]
    file=open('city_data.txt')
#We Loop through the file so that the contents in the file is extracted line by line.
    for i in file:
        x=i.split()
#The lines in file are string by default so we use split method to split the string into a list
        x=[x[0],float(x[1]),float(x[2])]
        city_list.append(x) 
#The distances in the list is stored in the type of string so we change the data type to float explicity and 
#then it is appened to the final list

    return city_list


## Calculation of the distance between two cities
- Distance between two points in coordinate geometry is calculated by the formula d = √[(x2 − x1)2 + (y2 − y1)2], where (x1, y1) and (x2, y2) are two points on the coordinate plane( in our case the cordinates of the particualr city)
- We need to use the square and square root to compute the distance formula, which is present in the built in math library(math).

In [3]:
def city_distance(city_list): 
    total_distance=0    
    for i in range(len(city_list)-1):
        first_city=city_list[i]
        second_city=city_list[i+1]
        distance=math.sqrt(math.pow(second_city[1]-first_city[1],2) + math.pow(second_city[2]-first_city[2],2))
        total_distance+=distance
        
#We got the distance of the city from the first element in the list till last element,
#but we need the distance between the first and last city also.To reach back to the city where the salesman started the journey.
  
    first_city=city_list[0] #first city
    last_city=city_list[-1] #last city
    distance=math.sqrt(math.pow(last_city[1]-first_city[1],2) + math.pow(last_city[2]-first_city[2],2))
    total_distance+=distance
    
    return total_distance



## Creation/selection of population

Selection involves selecting individuals from the current population to be parents based on their fitness. It is based on the idea that individuals with better fitness (i.e., those who are closer to the optimal solution) should have a higher chance of being selected.


In [4]:
# selecting the population
#here it create permuations of solution based on the size we pass,and return the distance 
def generate_population(city_list, size):
    population = []
    for i in range(size):
        new_city_list= city_list.copy()
        random.shuffle(new_city_list)
        new_city_distance= city_distance(new_city_list)
        population.append([new_city_distance, new_city_list])
    return population

In [5]:
def genetic_algorithm(population,lenCities,tournament_size,mutation_rate,crossover_rate):
    gen_number = 0
    for i in range(200):
        new_population = []

#Elitism-selection of the two best solutions
        new_population.append(sorted(population)[0])
        new_population.append(sorted(population)[1])

        for i in range(int((len(population) - 2)/ 2)):
            # crossover and selection operation is performed
            rn = random.random()
            if rn< crossover_rate:
                genome_parent1 = sorted(random.choices(population, k=tournament_size))[0]

                genome_parent2 = sorted(random.choices(population, k=tournament_size))[0]

                point = random.randint(0, lenCities - 1)

                genome_child1 = genome_parent1[1][0:point]
                for j in genome_parent2[1]:
                    if j not in genome_child1:
                        genome_child1.append(j)

                genome_child2 = genome_parent2[1][0:point]
                for j in genome_parent1[1]:
                    if j not in genome_child2:
                        genome_child2.append(j)

            # if the crossover fails,then we take any of the 2 solutions as the 2 child
            else:
                genome_child1 = random.choices(population)[0][1]
                genome_child2 = random.choices(population)[0][1]

            # mutation operation
            if random.random() < mutation_rate:
                point1 = random.randint(0, lenCities - 1)
                point2 = random.randint(0, lenCities - 1)
                genome_child1[point1], genome_child1[point2] = (genome_child1[point2],genome_child1[point1])

                point1 = random.randint(0, lenCities - 1)
                point2 = random.randint(0, lenCities - 1)
                genome_child2[point1], genome_child2[point2] = genome_child2[point2],genome_child2[point1]
                #used to swap the gene from child 2 at different place
            new_population.append([city_distance(genome_child1), genome_child1])
            new_population.append([city_distance(genome_child2), genome_child2])

        population = new_population

        gen_number += 1
        
        print("The shortest distance in the generation number",gen_number,"is",sorted(population)[0][0])



    result = sorted(population)[0]

    return result, gen_number



In [6]:
def construct_map(city_list, shortest_path):

    # To plot each cities x and y cordinate and annotate them
    for location in city_list:
        city_name,long,lat=location[0],location[1],location[2]
        plt.plot(long, lat,'o') 
        plt.annotate(city_name,(long, lat))

    for i in range(len(shortest_path[1])-1):
 
        city_left = shortest_path[1][i]
        city_right = shortest_path[1][i + 1]

        plt.plot([city_left[1], city_right[1]], [city_left[2], city_right[2]],"green")

    first_city = shortest_path[1][0]
    last_city = shortest_path[1][-1]
    plt.plot([first_city[1], last_city[1]], [first_city[2], last_city[2]],"red")

    plt.show()
    

In [7]:

pop_size = 1000
tournament_size = 5
mutation_rate = 0.2
crossover_rate = 0.8
city_names=[]

city_list = load_city()
firstPopulation= generate_population(city_list, pop_size)
best_solution,genNumber = genetic_algorithm(firstPopulation,len(city_list),tournament_size,mutation_rate,crossover_rate)
    
#we have a list containing the best solution
#we now split that list into 2 seperate variable(shortest_route and shortest_path)
shortest_distance=best_solution[0]
shortest_route=best_solution[1]
    
for p in range(len(shortest_route)):
    city_names.append(shortest_route[p][0])
    

print("##############################################")
print("Final Generation: ",str(genNumber))
print("##############################################")    
print("Shortest distance applying genetic alogorithm ->",str(shortest_distance))
print("Shortest route applying genetic alogorithm ->",city_names)

print("##############################################")
    
construct_map(city_list, best_solution)




FileNotFoundError: [Errno 2] No such file or directory: 'city_data.txt'